In [1]:
!pip install -U git+https://github.com/huggingface/transformers -q

In [2]:
!wget https://d8it4huxumps7.cloudfront.net/files/66e31d6ee96cd_student_resource_3.zip
!unzip 66e31d6ee96cd_student_resource_3.zip
!rm 66e31d6ee96cd_student_resource_3.zip

--2024-09-14 16:36:33--  https://d8it4huxumps7.cloudfront.net/files/66e31d6ee96cd_student_resource_3.zip
Resolving d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)... 18.160.249.126, 18.160.249.41, 18.160.249.9, ...
Connecting to d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)|18.160.249.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5184192 (4.9M) [application/zip]
Saving to: '66e31d6ee96cd_student_resource_3.zip'

66e31d6ee96cd_stude 100%[===================>]   4.94M  14.0MB/s    in 0.4s    

2024-09-14 16:36:34 (14.0 MB/s) - '66e31d6ee96cd_student_resource_3.zip' saved [5184192/5184192]

Archive:  66e31d6ee96cd_student_resource_3.zip
   creating: student_resource 3/
  inflating: __MACOSX/._student_resource 3  
  inflating: student_resource 3/sample_code.py  
  inflating: __MACOSX/student_resource 3/._sample_code.py  
  inflating: student_resource 3/.DS_Store  
  inflating: __MACOSX/student_resource 3/._.DS_Store  
   creating: s

In [3]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import pandas as pd
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO
import os
from tqdm import tqdm


In [5]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

english = {}


for k in entity_unit_map.keys():
    english[k] = k

english["item_weight"] = "weight"
english["item_volume"] = "volume"


# Setup

In [6]:
import pandas as pd
df = pd.read_csv("/kaggle/working/student_resource 3/dataset/test.csv")

In [50]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

<h1> Download images parallely </h1>


In [53]:
subset = df[:10]

print(subset["image_link"])

0    https://m.media-amazon.com/images/I/110EibNycl...
1    https://m.media-amazon.com/images/I/11TU2clswz...
2    https://m.media-amazon.com/images/I/11TU2clswz...
3    https://m.media-amazon.com/images/I/11TU2clswz...
4    https://m.media-amazon.com/images/I/11gHj8dhhr...
5    https://m.media-amazon.com/images/I/11gHj8dhhr...
6    https://m.media-amazon.com/images/I/11gHj8dhhr...
7    https://m.media-amazon.com/images/I/11lshEUmCr...
8    https://m.media-amazon.com/images/I/21+i52HRW4...
9    https://m.media-amazon.com/images/I/21-LmSmehZ...
Name: image_link, dtype: object


In [52]:
!pip install constants

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
import re
import constants
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)
    print(image_save_path)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)


In [56]:
import os
os.makedirs("downloaded_images", exist_ok=True)


save_dir = "downloaded_images"
image_urls = subset['image_link'].tolist()

download_images(image_urls, save_dir, allow_multiprocessing=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

downloaded_images/11TU2clswzL.jpgdownloaded_images/110EibNyclL.jpgdownloaded_images/11TU2clswzL.jpg

downloaded_images/11gHj8dhhrL.jpgdownloaded_images/21-LmSmehZL.jpgdownloaded_images/11TU2clswzL.jpgdownloaded_images/11gHj8dhhrL.jpgdownloaded_images/11lshEUmCrL.jpg
downloaded_images/11gHj8dhhrL.jpg



downloaded_images/21+i52HRW4L.jpg




100%|██████████| 10/10 [00:00<00:00, 159.06it/s]
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Inference

In [57]:
transform = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.ToTensor(),
])


In [58]:
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def url_to_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    resized = transform(img)
    return resized

# Replace the 'image_link' column with PIL.Image objects


In [59]:
def get_message(entity_name):
    message = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                    },
                    {"type": "text", "text": f"What is the {english[entity_name]} of the item in the image"},
                ],
            }
        ]
    return message

In [60]:
from typing import List
import torch
from tqdm import tqdm

def forward(entity_names: List[str], images: List[torch.Tensor], batch_size: int = 4):
    """
    Parv ka batch size wala logic update kiya hai thoda ( usne nahi banaya tha )
    """

    # Split data into smaller sub-batches
    num_samples = len(entity_names)
    output_text = []

    for i in tqdm(range(0, num_samples, batch_size)):
        # sub-batch
        sub_batch_entity_names = entity_names[i:i + batch_size]
        sub_batch_images = images[i:i + batch_size]


        messages = [get_message(entity) for entity in sub_batch_entity_names]

        texts = [
            processor.apply_chat_template(msg, add_generation_prompt=True)
            for msg in messages
        ]


        inputs = processor(
            text=texts,
            images=sub_batch_images,
            padding=True,
            return_tensors="pt"
        )

        inputs = inputs.to("cuda")

        # Generate output tokens
        output_ids = model.generate(**inputs, max_new_tokens=32)

        # Extract only generated tokens (remove the input prompt)
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(inputs.input_ids, output_ids)
        ]

        # Decode the generated IDs into text
        sub_batch_output_text = processor.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )

        # Append the output from this sub-batch to the final result
        output_text.extend(sub_batch_output_text)

        # Free GPU memory
        del inputs
        del output_ids
        del generated_ids
        torch.cuda.empty_cache()  # Clear cache to free memory

    return output_text

In [61]:
def get_local_filename(image_link):
    # Extract just the filename from the URL
    return os.path.basename(image_link)

In [62]:
imgs = []
enames = []


In [63]:
for ix, row in subset.iterrows():
    image_link = row["image_link"]
    local_filename = get_local_filename(image_link)
    img_path = os.path.join("downloaded_images", local_filename)

    if os.path.exists(img_path):
        # Open and transform the image
        with Image.open(img_path).convert('RGB') as img:
            img_tensor = transform(img)
        imgs.append(img_tensor)

        ename = row["entity_name"]
        enames.append(ename)
    else:
        print(f"Warning: File not found: {img_path}")

In [64]:
import time
import torch

images_tensor = torch.stack(imgs)

start = time.time()
output = forward(enames, imgs)
end = time.time()

100%|██████████| 3/3 [00:26<00:00,  8.87s/it]


In [65]:
execution_time = end - start  # Calculate the difference
print(f"Execution time: {execution_time} seconds")


Execution time: 26.608604192733765 seconds


In [66]:
output

["I'm sorry, but I cannot provide an answer to your question about the height of the item in the image as there is no image provided. Please upload an",
 'The width of the item in the image is 1000 pixels.',
 "I'm sorry, but the image you provided is too dark and unclear to determine the height of the item. Can you please provide more information or a clearer image",
 "I'm sorry, but I cannot provide an answer to your question about the depth of the item in the image as there is no image provided. Please upload an",
 "I'm sorry, but I cannot provide an answer to your question about the depth of the item in the image as there is no image provided. Please upload an",
 "I'm sorry, but the image you provided is too dark and unclear to determine the height of the item. Can you please provide more information or a clearer image",
 'The width of the item in the image is 100 pixels.',
 'The height of the item in the image is approximately 100 pixels.',
 'The width of the item in the image is 1

In [71]:
import re

# Dictionary to map unit abbreviations to full forms
unit_mapping = {
    'g': 'gram',
    'kg': 'kilogram',
    'mg': 'milligram',
    'ug': 'microgram',
    'lb': 'pound',
    'oz': 'ounce',
    'ton': 'ton',
    'mm': 'millimeter',
    'cm': 'centimeter',
    'm': 'meter',
    'ft': 'foot',
    'in': 'inch',
    'yd': 'yard'
}

# Function to convert the sentence
def extract_measurement(sentence):
    # Regular expression to match number and unit
    match = re.search(r'(\d+(\.\d+)?)\s*(\w+)', sentence)

    if match:
        number = match.group(1)
        unit = match.group(3)

        # Convert unit to full form
        full_unit = unit_mapping.get(unit, unit)

        # Form the final string
        result = f"{number} {full_unit}"
        return result
    else:
        return None

# Example usage
sentence_1 = "The weight of the ball is 200 g."
sentence_2 = "The width is 5mm."

print(extract_measurement(sentence_1))  # Output: 200 gram
print(extract_measurement(sentence_2))  # Output: 5 millimeter

200 gram
5 millimeter


In [72]:
for x in output:

    predicted_measurement = extract_measurement(x)

    if predicted_measurement:
        print(f"Predicted: {predicted_measurement}")
    else:
        print(f"Predicted: [No measurement found]")

Predicted: [No measurement found]
Predicted: 1000 pixels
Predicted: [No measurement found]
Predicted: [No measurement found]
Predicted: [No measurement found]
Predicted: [No measurement found]
Predicted: 100 pixels
Predicted: 100 pixels
Predicted: 1000 pixels
Predicted: [No measurement found]


In [73]:
import csv
import re
import random

def formating_csv(text_input):
    # Define mapping from short form to full form
    unit_full_form_mapping = {
        'cm': 'centimeter',
        'ft': 'foot',
        'yd': 'yard',
        'm': 'meter',
        'mm': 'millimeter',
        'in': 'inch',
        'g': 'grams',
        'kg': 'kilogram',
        'mg': 'milligram',
        'µg': 'microgram',
        'oz': 'ounce',
        'lb': 'pound',
        't': 'ton',
        'kV': 'kilovolt',
        'V': 'volt',
        'mV': 'millivolt',
        'kW': 'kilowatt',
        'W': 'watt',
        'cL': 'centiliter',
        'L': 'liter',
        'mL': 'milliliter',
        'µL': 'microliter',
        'dL': 'deciliter',
        'cup': 'cup',
        'pt': 'pint',
        'in^3': 'cubic inch',
        'ft^3': 'cubic foot',
        'gal': 'gallon',
        'imp gal': 'imperial gallon',
        'fl oz': 'fluid ounce',
        'qt': 'quart',
        'KG': 'kilogram',
        'gram': 'grams'
    }
    
    random_values = [10, 100, 20, 200, 50, 300, 250]

    # Prepare data for CSV
    data = []
    index = 0

    # Regex pattern to extract numeric value and unit from the text
    pattern = r'(\d+)\s*([a-zA-Z^3]+)'

    # Process each input line
    for line in text_input:
        if "[No measurement found]" in line:
            # If no measurement is found, substitute with a random value (e.g., "10 grams")
            random_value = random.randint(5, 20)  # Random value between 5 and 20
            prediction = f"{random_value} grams"
        else:
            # Extract value and unit using regex
            match = re.search(pattern, line)
            if match:
                value, short_unit = match.groups()
                # Convert the short unit to its full form
                full_unit = unit_full_form_mapping.get(short_unit, short_unit)  # Default to the same unit if not found
                prediction = f"{value} {full_unit}"
            else:
                # If no match, set prediction as None or default
                random_value = random.choice(random_values)
                prediction = f"{random_value} grams"

        # Append the prediction to the data list
        data.append({"index": index, "prediction": prediction})
        index += 1

    # Define the CSV file name
    csv_file = "text_input_predictions.csv"

    # Write to CSV file
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=["index", "prediction"])
        writer.writeheader()
        writer.writerows(data)

    print(f"CSV file '{csv_file}' created successfully.")
    with open(csv_file, 'r') as file:
        df = file.read()
        print(df)

    return df


In [74]:
de = formating_csv(output)
de

CSV file 'text_input_predictions.csv' created successfully.
index,prediction
0,250 grams
1,1000 pixels
2,20 grams
3,200 grams
4,20 grams
5,200 grams
6,100 pixels
7,100 pixels
8,1000 pixels
9,250 grams



'index,prediction\n0,250 grams\n1,1000 pixels\n2,20 grams\n3,200 grams\n4,20 grams\n5,200 grams\n6,100 pixels\n7,100 pixels\n8,1000 pixels\n9,250 grams\n'